<a href="https://colab.research.google.com/github/santiagoprado12/Tesis/blob/main/Implementacion_tecnicas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Procesamiento inicial

In [ ]:
import datetime 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.offline as py
import plotly.graph_objects as go

In [ ]:
#mainpath="/Users/santi/OneDrive/Documents"
#filename="Casos diarios y Beta.csv"
#fullpath=mainpath+"/"+filename
casos_diarios_DF=pd.read_csv("Casos diarios y Beta.csv")
#casos_diarios_DF=pd.read_csv(fullpath)


In [ ]:
casos_diarios_DF["enumerado"]=list(range(0,len(casos_diarios_DF["casos por dia"])))

In [ ]:
casos_diarios_DF

,fechas,casos por dia,fallecidos por dia,recuperados por dia,sintomaticos activos,expuestos,acumulado muertes,acumulado recuperados,acumulado casos,acumulado sintomaticos,acumulado expuestos,activos por dia,asintomaticos,Gamma,Beta,Sigma,BetaSEIR,enumerado
0,2020-03-18,139,0,0,138,16,1,4,768,754,1244,763,9,0.047619,0.214979,0.141823,0.234668,0
1,2020-03-19,162,0,1,160,-12,1,4,907,892,1260,902,10,0.047619,0.211859,0.164904,0.199767,1
2,2020-03-20,198,0,1,195,-33,1,5,1069,1052,1248,1063,11,0.047619,0.217641,0.203440,0.188397,2
3,2020-03-21,160,0,3,156,-46,1,6,1267,1247,1215,1260,13,0.047619,0.165054,0.184229,0.130562,3
4,2020-03-22,152,1,3,142,-3,1,9,1427,1403,1169,1417,14,0.047619,0.146968,0.182041,0.141889,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313,2021-01-25,8024,387,17813,-6550,-7056,52658,1812618,2131290,237659,76693,266014,28355,0.047619,0.009013,0.063068,-0.010922,313
314,2021-01-26,7307,356,16660,-9946,-3339,53045,1830431,2139314,231109,69637,255838,24729,0.047619,0.009345,0.012076,-0.011667,314
315,2021-01-27,6797,333,16583,-10051,-3277,53401,1847091,2146621,221163,66298,246129,24966,0.047619,0.005900,0.003703,-0.014724,315
316,2021-01-28,6440,316,16404,-11297,-2006,53734,1863674,2153418,211112,63021,236010,24898,0.047619,0.003229,-0.024439,-0.017959,316


In [ ]:
#casos_diarios_DF=casos_diarios_DF.set_index('enumerado')
#casos_diarios_DF["enumerado"]=list(range(0,len(casos_diarios_DF["casos por dia"])))

In [ ]:
#casos_diarios_DF

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=casos_diarios_DF["fechas"],
    y=casos_diarios_DF["casos por dia"],
    name="Casos diarios"
))

fig.add_trace(go.Scatter(
    x=casos_diarios_DF["fechas"],
    y=casos_diarios_DF["fallecidos por dia"],
    name="Muertes diarias"
))

fig.add_trace(go.Scatter(
    x=casos_diarios_DF["fechas"],
    y=casos_diarios_DF["recuperados por dia"],
    name="Recuperaciones diarias"
))

fig.add_trace(go.Scatter(
    x=casos_diarios_DF["fechas"],
    y=casos_diarios_DF["activos por dia"],
    name="Activos por dia"
))

fig.update_layout(title="Casos nuevos por dia")

fig.update_layout(showlegend=True)

fig.show()

# SEPARAR LOS DATOS


In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model
from sklearn.metrics import *
from sklearn.model_selection import TimeSeriesSplit

In [ ]:
X=casos_diarios_DF["enumerado"]
Y=casos_diarios_DF["activos por dia"]

In [ ]:
def split_data(test_data=1, step=1, datos=None, train_min=1):
  n=train_min
  i=1
  train_index=[]
  test_index=[]
  while (test_data+n)<(len(datos)):
    train_index.append(list(range(0,n)))
    test_index.append(list(range(n,n+test_data)))
    n+=step
  train_index.append(list(range(0,len(datos)-test_data)))
  test_index.append(list(range(len(datos)-test_data,len(datos))))
  return train_index, test_index

In [ ]:
train_index, test_index=split_data(test_data=20, step=7, datos=X, train_min=30)

In [ ]:
fig = go.Figure()

for i,test in enumerate(test_index):

  fig.add_trace(go.Scatter(
      x=test,
      y=np.ones(len(test))*i,
      name="Test "+str(i)
  ))

for i,train in enumerate(train_index):

  fig.add_trace(go.Scatter(
      x=train,
      y=np.ones(len(train))*i,
      name="Train "+str(i)
  ))

fig.update_layout(title="Separacion de los datos")

fig.update_layout(showlegend=True)

fig.show()

# REGRESION LINEAL

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model
from sklearn.metrics import *
from sklearn.model_selection import TimeSeriesSplit

In [ ]:
X=casos_diarios_DF["enumerado"]
Y=casos_diarios_DF["activos por dia"]

In [ ]:
valores=[2,3,4,5,6,7,8]
for n in valores:  
  fig = go.Figure()
  fig_score=go.Figure()
  score=[]
  Poly= PolynomialFeatures(degree=n)
  g=0
  fig.add_trace(go.Scatter(
      x=X,
      y=casos_diarios_DF["activos por dia"],
      name="Real"
  ))
  score=[]
  for i,test in enumerate(test_index):
    X_train, X_test = X[train_index[i]], X[test_index[i]]
    Y_train, Y_test = Y[train_index[i]], Y[test_index[i]]

    
    X_data_train = Poly.fit_transform(np.array(X_train)[:,np.newaxis])
    X_data_test = Poly.fit_transform(np.array(X_test)[:,np.newaxis])

    lm=linear_model.LinearRegression()
    lm.fit(X_data_train, Y_train)

    lis_train=[]
    lis_test=[]
    for i,l in enumerate(lm.coef_):
      lis_test.append(l*(X_test**i))
      lis_train.append(l*(X_train**i))
    
    Yp_train=lm.intercept_+sum(lis_train)
    Yp_test=lm.intercept_+sum(lis_test)

    fig.add_trace(go.Scatter(
      x=X_train,
      y=Yp_train,
      name="Train"+str(g)
    ))
    fig.add_trace(go.Scatter(
      x=X_test,
      y=Yp_test,
      name="Test"+str(g)
    ))
    g+=1
    score.append(np.sqrt(mean_squared_error(Y_test,Yp_test)))

  fig.update_layout(title="Casos nuevos por dia")

  fig.update_layout(showlegend=True)

  fig.show()
    
  fig_score.add_trace(go.Scatter(
      x=list(range(len(test_index))),
      y=score,
      name="Train"+str(g)
  ))
  fig_score.update_layout(title="errores")

  fig_score.update_layout(showlegend=True)

  fig_score.show()
  print("El promedio es:")
  print(np.mean(score))
  print("La desviacion estandar es:")
  print(np.sqrt(np.var(score)))

El promedio es:
100238.27352625418
La desviacion estandar es:
96063.3359007771


# SUPPORT VECTOR REGRESSION





In [ ]:
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler

In [ ]:
X=casos_diarios_DF["enumerado"]
Y=casos_diarios_DF["activos por dia"]

In [ ]:
sc_X = StandardScaler()
sc_Y = StandardScaler()

In [ ]:
epsilon=[0.5,0.3,0.1,0.05,0.01,0.005,0.001,0.00001]
C_parameter=[1,2,4,8]
for ep in epsilon:
  for c in C_parameter:
    fig = go.Figure()
    fig_score=go.Figure()
    score=[]
    g=0
    fig.add_trace(go.Scatter(
        x=X,
        y=casos_diarios_DF["activos por dia"],
        name="Real"
    ))
    score=[]

    X_data=sc_X.fit_transform(np.array(X[:,np.newaxis]))
    Y_data=sc_Y.fit_transform(np.array(Y[:,np.newaxis]))

    for i,test in enumerate(test_index):
      X_train, X_test = X_data[train_index[i]], X_data[test_index[i]]
      Y_train, Y_test = Y_data[train_index[i]], Y_data[test_index[i]]

      regressor = SVR(kernel='rbf', C=c, epsilon=ep)
      regressor.fit(np.array(X_train),Y_train.ravel())

      y_pred = regressor.predict(X_train)
      y_pred = sc_Y.inverse_transform(y_pred)
      y_predt = regressor.predict(X_test)
      y_predt = sc_Y.inverse_transform(y_predt)
      x = sc_X.inverse_transform(X_train)
      xt = sc_X.inverse_transform(X_test)
      yt= sc_Y.inverse_transform(Y_test)
      fig.add_trace(go.Scatter(
        x=x.ravel(),
        y=y_pred,
        name="Train"+str(g)
      ))
      
      fig.add_trace(go.Scatter(
        x=xt.ravel(),
        y=y_predt,
        name="Test"+str(g)
      ))
      g+=1
      score.append(np.sqrt(mean_squared_error(yt,y_predt)))

    fig.update_layout(title="Casos nuevos por dia")

    fig.update_layout(showlegend=True)

    fig.show()
      
    fig_score.add_trace(go.Scatter(
        x=list(range(len(test_index))),
        y=score,
        name="Train"+str(g)
    ))
    fig_score.update_layout(title="errores")

    fig_score.update_layout(showlegend=True)

    fig_score.show()

    print(f"Epsilon: {ep} y C: {c}")
    print("El promedio es:")
    print(np.mean(score))
    print("La desviacion estandar es:")
    print(np.sqrt(np.var(score)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: FutureWarning:

Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning:

Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.



Epsilon: 0.001 y C: 1
El promedio es:
31786.829366562575
La desviacion estandar es:
18292.487600724533


# SIR

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from scipy.optimize import minimize

In [ ]:
def deriv(y, t, N, beta, gamma):
    S, I, R = y
    dSdt = -beta * S * I / N
    dIdt = beta * S * I / N - gamma * I
    dRdt = gamma * I
    return dSdt, dIdt, dRdt

In [ ]:
df=pd.DataFrame()
df['Infected']=casos_diarios_DF['activos por dia']
df['Recovered']=casos_diarios_DF['acumulado recuperados']
df['Deaths']=casos_diarios_DF['acumulado muertes']

In [ ]:
df['Removed'] = df['Recovered']+df['Deaths']
df['Beta']=casos_diarios_DF['Beta']

In [ ]:
df['index']=list(range(0,len(df)))
df=df.set_index('index')

In [ ]:
def sir(beta):
  res=[]
  N= 48200000
  the_gamma = 1/21
  init_index=0

  I_a=df['Infected'][0]
  res=[I_a]
  R_a=df['Removed'][0]
  S_a=N-I_a-R_a

  for i,be in enumerate(beta):
    t = np.linspace(0,1,2)
    y0 = S_a,I_a,R_a
    S,I,R = odeint(deriv, y0, t, args=(N, be, the_gamma)).T
    I_a=I[1]
    R_a=R[1]
    S_a=S[1]
    res.append(I[1])
  return res

In [ ]:
def grafica(x, g, y, z, d):
  
  fig.add_trace(go.Scatter(
      x=x,
      y=y,
      name="SIR con"+str(d)
  ))
  fig.add_trace(go.Scatter(
      x=g,
      y=z,
      name="SIR predicho con"+str(d)
  ))
    
  fig.update_layout(title="Real vs SIR")

  fig.update_layout(showlegend=True)

In [ ]:
def pred_beta(beta_train, beta_test, datos_atras):
  d=datos_atras*-1
  beta_new=beta_train[:-1]
  for n in range(0,len(beta_test)):
    beta_new.append(np.mean(beta_new[d:]))

  return beta_new


In [ ]:
X=casos_diarios_DF["enumerado"]
Y=casos_diarios_DF["Beta"]
I=df['Infected']
R=df['Removed']
A=casos_diarios_DF["activos por dia"]

In [ ]:
datos_atras=[1,2,3,4,5,6,7,8,9,10,11,12]
for dat in datos_atras:
  fig = go.Figure()
  score=[]
  for i,test in enumerate(test_index):
    X_train, X_test = X[train_index[i]], X[test_index[i]]
    Beta_train, Beta_test = Y[train_index[i]], Y[test_index[i]]
    Activos_train, Activos_test = A[train_index[i]], A[test_index[i]]

    betaa=pred_beta(list(Beta_train), list(Beta_test), dat)
    res=sir(betaa)
    
    rf=pd.DataFrame()
    rf['res']=res
    re=rf['res']
    predict_train, predict_test = re[train_index[i]], re[test_index[i]]
    score.append(np.sqrt(mean_squared_error(Activos_test,predict_test)))
    
    grafica(X_train,X_test,predict_train,predict_test,dat)


  fig.add_trace(go.Scatter(
      x=casos_diarios_DF["enumerado"],
      y=casos_diarios_DF["activos por dia"],
      name="Real"
  ))

  fig.show()

  fig_score = go.Figure()
  fig_score.add_trace(go.Scatter(
      x=list(range(len(test_index))),
      y=score,
      name="Train"+str(g)
  ))
  fig_score.update_layout(title="errores")

  fig_score.update_layout(showlegend=True)

  fig_score.show()
  print("El promedio es:")
  print(np.mean(score))
  print("La desviacion estandar es:")
  print(np.sqrt(np.var(score)))


El promedio es:
25624.805794216045
La desviacion estandar es:
27025.069917676752


# SEIR

In [ ]:
def derivSEIR(y, t, N, beta, gamma, sigma):
    S, E, I, R = y
    dSdt = -beta * S * I / N
    dEdt = beta * S * I / N - sigma * E
    dIdt = sigma * E - gamma*I
    dRdt = gamma * I
    return dSdt, dEdt, dIdt, dRdt 

In [ ]:
def pred_beta(beta_train, beta_test, datos_atras):
  d=datos_atras*-1
  beta_new=beta_train[:-1]
  for n in range(0,len(beta_test)):
    beta_new.append(np.mean(beta_new[d:]))

  return beta_new
def pred_sigma(sigma_train, sigma_test, datos_atras):
  d=datos_atras*-1
  sigma_new=sigma_train[:-1]
  for n in range(0,len(sigma_test)):
    sigma_new.append(np.mean(sigma_new[d:]))

  return sigma_new


In [ ]:
def seir(beta,sigma):
  res1=[]
  N= 48200000
  init_index=0
  the_gamma = 1/21
  I_a=df['InfectedSEIR'][0]
  res1=[I_a]
  Ex_a=df['Expuestos'][0]
  R_a=df['Removed'][0]
  S_a=N-I_a-R_a-Ex_a

  zip_beta_sigma=list(zip(sigma,beta))
  for si,be in zip_beta_sigma:
    t = np.linspace(0,1,2)
    y0 = S_a,Ex_a,I_a,R_a
    S,E,I,R = odeint(derivSEIR, y0, t, args=(N, be, the_gamma,si)).T
    I_a=I[1]
    Ex_a=E[1]
    R_a=R[1]
    S_a=S[1]
    res1.append(I[1])

  return res1

In [ ]:
 
def grafica(x, g, y, z, d):
  
  fig.add_trace(go.Scatter(
      x=x,
      y=y,
      #name="SIR con"+str(d)
  ))
  fig.add_trace(go.Scatter(
      x=g,
      y=z,
      #name="SIR predicho con"+str(d)
  ))
    
  fig.update_layout(title="Real vs SIR")

  fig.update_layout(showlegend=True)

In [ ]:
df=pd.DataFrame()
df['InfectedSEIR']=casos_diarios_DF['acumulado sintomaticos']
df['Expuestos']=casos_diarios_DF['acumulado expuestos']
df['Recovered']=casos_diarios_DF['acumulado recuperados']
df['Deaths']=casos_diarios_DF['acumulado muertes']
df['Removed'] = df['Recovered']+df['Deaths']
df['Beta']=casos_diarios_DF['BetaSEIR']
df['Sigma']=casos_diarios_DF['Sigma']

In [ ]:
X=casos_diarios_DF["enumerado"]
B=df['Beta']
S=df['Sigma']
AS=casos_diarios_DF['asintomaticos']

In [ ]:
datos_atras=[1,2,3,4,5,6,7,8,9,10,11,12]
for dat in datos_atras:
  fig = go.Figure()
  score=[]

  for i,test in enumerate(test_index):
    X_train, X_test = X[train_index[i]], X[test_index[i]]
    Beta_train, Beta_test = B[train_index[i]], B[test_index[i]]
    Sigma_train, Sigma_test = S[train_index[i]], S[test_index[i]]
    Activos_train, Activos_test = A[train_index[i]], A[test_index[i]]
    Asinto_train, Asinto_test = AS[train_index[i]], AS[test_index[i]]
    asintomatico=pd.concat([Asinto_train, Asinto_test], axis=0)

    betaa=pred_beta(list(Beta_train), list(Beta_test), dat)
    sigmaa=pred_sigma(list(Sigma_train), list(Sigma_test), dat)
    r1=seir(betaa,sigmaa)
    r11=asintomatico+r1

    rf1=pd.DataFrame()
    rf1["res"]=r11
    re=rf1["res"]
    predict_train, predict_test = re[train_index[i]], re[test_index[i]]
    score.append(np.sqrt(mean_squared_error(Activos_test,predict_test)))

    grafica(X_train,X_test,predict_train,predict_test,test)

  fig.add_trace(go.Scatter(
      x=casos_diarios_DF["enumerado"],
      y=casos_diarios_DF["activos por dia"],
      name="Real"
  ))

  fig.show()

  fig_score = go.Figure()
  fig_score.add_trace(go.Scatter(
      x=list(range(len(test_index))),
      y=score,
      name="Train"+str(g)
  ))
  fig_score.update_layout(title="errores")

  fig_score.update_layout(showlegend=True)

  fig_score.show()
  print("El promedio es:")
  print(np.mean(score))
  print("La desviacion estandar es:")
  print(np.sqrt(np.var(score)))

El promedio es:
17644.793977721514
La desviacion estandar es:
17147.59313192758


# LSTM